In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.chdir('drive/My Drive')
os.listdir()

['depthwise_conv2d.py',
 'Colab Notebooks',
 'logs',
 'Pickles',
 'Zips',
 'model.py',
 'data_preprocess.py',
 'utils.py',
 'CRoam Model Training  Report.pdf',
 'speaker_ver1_model.h5',
 'speaker_ver1_model.pb',
 'speaker_ver1a_model.h5',
 'speaker_ver1b_model.h5',
 'speaker_ver1a_model.pb',
 'graph_def_for_reference.pb.ascii',
 'speaker_ver1b_model.pb',
 'new_mix_help_train_x.pickle',
 'new_mix_help_train_y.pickle',
 'total_bachao_x_new.pickle',
 'total_bachao_y_new.pickle',
 'Data',
 'Bachao_Models',
 'Help_Models']

In [3]:
os.chdir('Bachao_Models')
os.listdir()

['HDF5', 'protobuf', 'newbachao.pb', 'graph_def_for_reference.pb.ascii']

In [10]:
!pip install tensorflow==1.15
# !pip install tensorflow==2.2.0

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 3.8MB 40.9MB/s 
     |████████████████████████████████| 512kB 46.5MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f6e54c14e8c2ca1d6cb95ed1dd3ba3477e4a46708385046d18aa13d5ea911bb1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version: ", tf.__version__)
print("Keras version: ", keras.__version__)

TensorFlow version:  2.4.1
Keras version:  2.4.0


In [5]:
'''
This script converts a .h5 Keras model into a Tensorflow .pb file.
Attribution: This script was adapted from https://github.com/amir-abdi/keras_to_tensorflow
MIT License
Copyright (c) 2017 bitbionic
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

import os
import os.path as osp
import argparse

import tensorflow as tf

from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def convertGraph( modelPath, outdir, numoutputs, prefix, name):
    '''
    Converts an HD5F file to a .pb file for use with Tensorflow.
    Args:
        modelPath (str): path to the .h5 file
           outdir (str): path to the output directory
       numoutputs (int):   
           prefix (str): the prefix of the output aliasing
             name (str):
    Returns:
        None
    '''
    
    #NOTE: If using Python > 3.2, this could be replaced with os.makedirs( name, exist_ok=True )
    if not os.path.isdir(outdir):
        os.mkdir(outdir)

    K.set_learning_phase(0)

    net_model = load_model(modelPath,compile=False)

    # Alias the outputs in the model - this sometimes makes them easier to access in TF
    pred = [None]*numoutputs
    pred_node_names = [None]*numoutputs
    for i in range(numoutputs):
        pred_node_names[i] = prefix+str(i)
        pred[i] = tf.identity(net_model.output[i], name=pred_node_names[i])
    print('Output nodes names are: ', pred_node_names)
    sess = K.get_session()
    
    # Write the graph in human readable
    f = 'graph_def_for_reference.pb.ascii'
    tf.train.write_graph(sess.graph.as_graph_def(), outdir, f, as_text=True)
    print('Saved the graph definition in ascii format at: ', osp.join(outdir, f))
    
    # Write the graph in binary .pb file
    from tensorflow.python.framework import graph_util
    from tensorflow.python.framework import graph_io
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    graph_io.write_graph(constant_graph, outdir, name, as_text=False)
    print('Saved the constant graph (ready for inference) at: ', osp.join(outdir, name))

'''
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--model','-m', dest='model', required=True, help='REQUIRED: The HDF5 Keras model you wish to convert to .pb')
    parser.add_argument('--numout','-n', type=int, dest='num_out', required=True, help='REQUIRED: The number of outputs in the model.')
    parser.add_argument('--outdir','-o', dest='outdir', required=False, default='./', help='The directory to place the output files - default("./")')
    parser.add_argument('--prefix','-p', dest='prefix', required=False, default='k2tfout', help='The prefix for the output aliasing - default("k2tfout")')
    parser.add_argument('--name', dest='name', required=False, default='output_graph.pb', help='The name of the resulting output graph - default("output_graph.pb")')
    args = parser.parse_args()

    convertGraph( args.model, args.outdir, args.num_out, args.prefix, args.name )
'''

'\nif __name__ == \'__main__\':\n\n    parser = argparse.ArgumentParser()\n    parser.add_argument(\'--model\',\'-m\', dest=\'model\', required=True, help=\'REQUIRED: The HDF5 Keras model you wish to convert to .pb\')\n    parser.add_argument(\'--numout\',\'-n\', type=int, dest=\'num_out\', required=True, help=\'REQUIRED: The number of outputs in the model.\')\n    parser.add_argument(\'--outdir\',\'-o\', dest=\'outdir\', required=False, default=\'./\', help=\'The directory to place the output files - default("./")\')\n    parser.add_argument(\'--prefix\',\'-p\', dest=\'prefix\', required=False, default=\'k2tfout\', help=\'The prefix for the output aliasing - default("k2tfout")\')\n    parser.add_argument(\'--name\', dest=\'name\', required=False, default=\'output_graph.pb\', help=\'The name of the resulting output graph - default("output_graph.pb")\')\n    args = parser.parse_args()\n\n    convertGraph( args.model, args.outdir, args.num_out, args.prefix, args.name )\n'

In [6]:
os.listdir()

['HDF5', 'protobuf', 'newbachao.pb', 'graph_def_for_reference.pb.ascii']

In [8]:
modelPath = 'HDF5/serverbachao1.h5'
net_model = load_model(modelPath,compile=False)
net_model.summary()

OSError: ignored

In [ ]:
import tensorflow.python.keras.backend as K
import tensorflow.compat.v1 as tf

In [ ]:
outdir = './'
numoutputs = 1
prefix = 'output'
name = 'serverbachao1.pb'
convertGraph(modelPath, outdir, numoutputs, prefix, name)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Output nodes names are:  ['output0']
Saved the graph definition in ascii format at:  ./graph_def_for_reference.pb.ascii
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


AssertionError: ignored

In [9]:
import tensorflow as tf
from tensorflow.python.platform import gfile
GRAPH_PB_PATH = 'help1.pb'
with tf.Session() as sess:
   print("loading graph....")
   with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
       graph_def = tf.GraphDef()
   graph_def.ParseFromString(f.read())
   sess.graph.as_default()
   tf.import_graph_def(graph_def, name='')
   graph_nodes=[n for n in graph_def.node]
   names = []
   for t in graph_nodes:
      names.append(t.name)
   print(names)

AttributeError: ignored

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(net_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

ImportError: ignored

In [ ]:
[n.name for n in tf.get_default_graph().as_graph_def().node][0]

In [ ]:
tf.get_default_graph()